In [ ]:
import threading
import time

class Garcom(threading.Thread):
    def __init__(self, nome, cozinha):
        super().__init__()
        self.nome = nome
        self.cozinha = cozinha

    def fazer_pedido(self, pedido):
        with self.cozinha.condicao:
            self.cozinha.pedidos_pendentes.append(pedido)
            print(f"{self.nome} fez o pedido {pedido}")
            self.cozinha.condicao.notify()

    def run(self):
        for i in range(10):
            self.fazer_pedido(i)
            time.sleep(0.5)  # Tempo simulado para fazer um pedido

        self.cozinha.garcom_concluido()

class Cozinheiro(threading.Thread):
    def __init__(self, nome, cozinha):
        super().__init__()
        self.nome = nome
        self.cozinha = cozinha

    def preparar_pedido(self):
        with self.cozinha.condicao:
            while not self.cozinha.pedidos_pendentes:
                self.cozinha.condicao.wait()

            pedido = self.cozinha.pedidos_pendentes.pop(0)
            print(f"{self.nome} preparou o pedido {pedido}")
            time.sleep(1.0)  # Tempo simulado para preparar o pedido

    def run(self):
        while not self.cozinha.garcoms_concluidos():
            self.preparar_pedido()

class Cozinha:
    def __init__(self, num_garcons):
        self.condicao = threading.Condition()
        self.pedidos_pendentes = []
        self.garcons_concluidos_count = 0
        self.num_garcons = num_garcons

    def garcom_concluido(self):
        with self.condicao:
            self.garcons_concluidos_count += 1
            if self.garcons_concluidos_count == self.num_garcons:
                self.condicao.notify_all()

    def garcoms_concluidos(self):
        with self.condicao:
            return self.garcons_concluidos_count == self.num_garcons

def main():
    num_garcons = 3
    num_cozinheiros = 1

    cozinha = Cozinha(num_garcons)

    garcons = []
    for i in range(num_garcons):
        garcom = Garcom(f"Garçom {i+1}", cozinha)
        garcom.start()
        garcons.append(garcom)

    cozinheiros = []
    for i in range(num_cozinheiros):
        cozinheiro = Cozinheiro(f"Cozinheiro {i+1}", cozinha)
        cozinheiro.start()
        cozinheiros.append(cozinheiro)

    start_time = time.time()

    for garcom in garcons:
        garcom.join()

    for cozinheiro in cozinheiros:
        cozinheiro.join()

    end_time = time.time()

    total_time = end_time - start_time
    print(f"Tempo total de atendimento: {total_time} segundos")

if __name__ == '__main__':
    main()
